# Summary

Make training and dev and test datasets from a set of files containing reading sessions


Required packages:


### Imports etc.

In [1]:
import copy
import datetime
import math
import random
import os,sys
import json
import time
print('ready')

ready


In [2]:
from multiprocessing import Pool


## Sessions

In [7]:
## select time period
day_start = datetime.date(2020,4,1)
day_end = datetime.date(2020,4,8)
date_array = \
    (day_start + datetime.timedelta(days=x) for x in range(0, (day_end-day_start).days))

## select language
# lang = 'wikidata'
lang = 'enwiki'

## select size of subsample (for wikidata there are ~50M sessions/day which might be too much), 
mult = 100000 ## in multiples of 100k
list_N_size = [1,2,1000,-1] ## -1 indicates the full dataset

## split into train-dev-test
p_train, p_dev,p_test = 0.8,0.1,0.1 ## has to sum to 1


PATH_IN = '/home/mgerlach/REPOS/reader-embedding/output/reading-sessions-filtered/'
PATH_OUT = '/home/mgerlach/REPOS/reader-embedding/output/reading-sessions-corpora/'
## get all files from reading sessions
list_filenames = []
for date_object in date_array:
    day_str = date_object.strftime("%Y-%m-%d")#str(datetime.date(year,month,day))
    filename = os.path.join(
        PATH_IN,
       'reading-sessions-filtered_%s_%s'%(lang,day_str)
    )
    if os.path.isfile(filename):
        list_filenames += [filename]
print('Number of files: ',len(list_filenames))


## use multiple cores to read different files (at most 10)
n_cores = min(10,len(list_filenames))

Number of files:  1


## get total number of sessions in all files

In [8]:
## get the number of sessions across all datasets
def read_sessions_from_file(filename_in):
    N_kept = 0
    for line in open(filename_in):
        N_kept+=1
    return N_kept

try:
    pool = Pool(n_cores)
    list_N_kept = pool.map(read_sessions_from_file,list_filenames)
finally:
    pool.close()
    pool.join()
## number of total lines
N=sum(list_N_kept)
print('Total number of sessions: ',N)

Total number of sessions:  24150756


## create randomly sampled corpora of different sizes

In [9]:
def sample_sessions_from_file(filename_in):
    with open(filename_tmp,'a') as fout:
        for line in open(filename_in):
            write_line = True
            ## write the line with some sampling probability
            if p_sample<1.:
                if random.random()>p_sample:
                    write_line=False
            if write_line==True:
                fout.write(line)

In [10]:
for N_size in list_N_size:
    if N_size<0:
        N_sel = N
    elif N_size*mult<=N:
        N_sel = N_size*mult
    else:
        continue
    print(N_size, N_sel)

        
    p_sample = min([1.1*N_sel/N,1.])
    filename_tmp = os.path.join(PATH_OUT,'tmp.corpus')
    with open(filename_tmp,'w') as fout:
        fout.write('')
    try:
        pool = Pool(n_cores)
        pool.map(sample_sessions_from_file,list_filenames)
    finally:
        pool.close()
        pool.join()
        
    
    filename_base = os.path.join(PATH_OUT,'%s_sample-%s'%(lang,N_sel))
    ## make sure we only have N_sel samples and write to corpus-file
    filename_out_corpus = filename_base+'.corpus'
    with open(filename_out_corpus,'w') as fout:
        i=0
        for line in open(filename_tmp):
            fout.write(line)
            i+=1
            if i==N_sel:
                break
                                      
    ## split into train-dev-test          
    filename_out_train = filename_base+'.train'
    filename_out_dev = filename_base+'.dev'
    filename_out_test = filename_base+'.test'

    N_train = int(N_sel*p_train)
    N_dev = int(N_sel*p_dev)
    i=0
    
    with open(filename_out_train,'w') as fout_train, open(filename_out_dev,'w') as fout_dev, open(filename_out_test,'w') as fout_test:
        for line in open(filename_out_corpus):
            if i<N_train:
                fout_train.write(line)
            elif i>=N_train and i<N_train+N_dev:
                fout_dev.write(line)
            elif i>=N_train+N_dev:
                fout_test.write(line)
            else:
                pass
            i+=1
                                 
    ## delete the tmp-file          
    os.system('rm -r %s'%filename_tmp)

1 100000
2 200000
-1 24150756
